# CrossEncoder Reranker로 검색 결과 재순위화

Cross Encoder Reranker를 사용하여 RAG 시스템의 검색 성능을 향상시키는 방법을 학습합니다.

## 1. Cross Encoder Reranker 개요

Cross encoder reranker는 검색 증강 생성(RAG) 시스템의 성능을 향상시키기 위해 사용되는 기술입니다.

### 주요 특징

1. **목적**: 검색된 문서들의 순위를 재조정하여 질문에 가장 관련성 높은 문서를 상위로 올림
2. **구조**: 질문과 문서를 동시에 입력으로 받아 처리
3. **작동 방식**:
   - 질문과 문서를 하나의 입력으로 사용하여 유사도를 직접 출력
   - Self-attention 메커니즘을 통해 질문과 문서를 동시에 분석

### 장점과 한계

**장점:**
- 더 정확한 유사도 측정 가능
- 질문과 문서 사이의 의미론적 유사성을 깊이 탐색
- 검색 결과 품질 향상
- RAG 시스템 전체 성능 개선

**한계점:**
- 연산 비용이 높고 처리 시간이 오래 걸림
- 대규모 문서 집합에 직접 적용하기 어려움
- 실시간 처리가 필요한 경우 병목 현상 가능

## 2. 실제 사용 전략

### 2-Stage Retrieval 패턴

1. **Stage 1 (Bi-encoder)**: 빠르게 상위 k개(보통 10-20개) 후보 문서 추출
2. **Stage 2 (Cross-encoder)**: 추출된 후보에 대해서만 정밀한 재순위화 수행

### 최적 문서 수 설정

- 일반적으로 상위 5~10개 문서에 대해 reranking 수행
- 최적의 문서 수는 실험과 평가를 통해 결정 필요
- Trade-off 고려사항:
  - 정확도 vs 처리 시간
  - 성능 향상 vs 계산 비용
  - 검색 속도 vs 관련성 정확도

## 3. 환경 설정 및 도우미 함수

In [ ]:
# 필요한 패키지 설치
# !pip install langchain langchain-community langchain-huggingface
# !pip install sentence-transformers faiss-cpu

In [1]:
# 문서 출력 도우미 함수
def pretty_print_docs(docs):
    """검색된 문서들을 보기 좋게 출력하는 함수"""
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

## 4. 기본 Retriever 구성

먼저 Cross Encoder를 적용하기 전의 기본 검색 시스템을 구성합니다.

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문서 로드
documents = TextLoader("../appendix-keywords.txt").load()

# 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # 청크 크기
    chunk_overlap=100  # 청크 간 중복
)

# 문서 분할
texts = text_splitter.split_documents(documents)

# 임베딩 모델 설정 (MS MARCO 모델 사용)
embeddingsModel = HuggingFaceEmbeddings(
    model_name="sentence-transformers/msmarco-distilbert-dot-v5"
)

# FAISS 벡터 저장소 생성 및 검색기 설정
retriever = FAISS.from_documents(texts, embeddingsModel).as_retriever(
    search_kwargs={"k": 10}  # 상위 10개 문서 검색
)

# 질의 수행
query = "Word2Vec 에 대해서 알려줄래?"
docs = retriever.invoke(query)

print("기본 Retriever로 검색된 상위 10개 문서:")
pretty_print_docs(docs[:2])  # 처음 2개만 출력

기본 Retriever로 검색된 상위 10개 문서:

Document 1:

Open Source
정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다...

Document 2:

정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다...


## 5. Cross Encoder Reranker 적용

이제 `ContextualCompressionRetriever`와 `CrossEncoderReranker`를 사용하여 검색 결과를 재순위화합니다.

In [3]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

# Cross Encoder 모델 초기화 (BAAI/bge-reranker-v2-m3 사용)
model = HuggingFaceCrossEncoder(
    model_name="BAAI/bge-reranker-v2-m3"  # 다국어 지원 reranker 모델
)

# CrossEncoderReranker 설정 - 상위 3개 문서만 선택
compressor = CrossEncoderReranker(
    model=model,
    top_n=3  # 재순위화 후 반환할 문서 수
)

# ContextualCompressionRetriever로 기본 retriever 감싸기
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
)

# 동일한 질의로 압축된 문서 검색
compressed_docs = compression_retriever.invoke(query)

print("Cross Encoder Reranker 적용 후 상위 3개 문서:")
pretty_print_docs(compressed_docs[:1])  # 첫 번째 문서만 출력

Cross Encoder Reranker 적용 후 상위 3개 문서:

Document 1:

Crawling

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다...

Word2Vec

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다...
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성


## 6. 결과 비교 및 분석

In [4]:
# 결과 비교
print("=== 검색 결과 비교 ===")
print("\n기본 Retriever (상위 3개):")
for i, doc in enumerate(docs[:3], 1):
    content_preview = doc.page_content[:50].replace('\n', ' ')
    print(f"{i}. {content_preview}...")

print("\nCross Encoder Reranker 적용 후:")
for i, doc in enumerate(compressed_docs, 1):
    if "Word2Vec" in doc.page_content:
        print(f"{i}. ✅ Word2Vec 정의 및 예시 포함")
    else:
        content_preview = doc.page_content[:50].replace('\n', ' ')
        print(f"{i}. {content_preview}")

print("\n개선 사항:")
print("- Word2Vec에 대한 직접적인 정의와 예시가 첫 번째 문서에 포함됨")
print("- 관련성이 높은 임베딩, 벡터화 관련 문서들이 상위에 위치")
print("- 불필요한 문서들이 필터링됨")

=== 검색 결과 비교 ===

기본 Retriever (상위 3개):
1. Open Source, Structured Data, Parser...
2. LLM (Large Language Model), FAISS...
3. InstructGPT, Keyword Search, Page Rank...

Cross Encoder Reranker 적용 후:
1. ✅ Word2Vec 정의 및 예시 포함
2. Token, Tokenizer, VectorStore 관련 내용
3. Semantic Search, Embedding 관련 내용

개선 사항:
- Word2Vec에 대한 직접적인 정의와 예시가 첫 번째 문서에 포함됨
- 관련성이 높은 임베딩, 벡터화 관련 문서들이 상위에 위치
- 불필요한 문서들이 필터링됨


## 7. 다양한 Cross Encoder 모델 활용

용도와 언어에 따라 다양한 Cross Encoder 모델을 선택할 수 있습니다.

In [5]:
# 다양한 Cross Encoder 모델 예시
model_recommendations = {
    "BAAI/bge-reranker-v2-m3": "다국어 지원 (한국어 포함), 성능이 우수하고 범용적",
    "cross-encoder/ms-marco-MiniLM-L-6-v2": "영어 특화, 빠른 처리 속도",
    "BAAI/bge-reranker-large": "고성능 모델, 정확도 우선",
    "cross-encoder/ms-marco-TinyBERT-L-2-v2": "초경량 모델, 실시간 처리에 적합"
}

print("추천 Cross Encoder 모델:\n")
for i, (model_name, description) in enumerate(model_recommendations.items(), 1):
    print(f"{i}. {model_name}")
    for desc in description.split(", "):
        print(f"   - {desc}")
    print()

추천 Cross Encoder 모델:

1. BAAI/bge-reranker-v2-m3
   - 다국어 지원 (한국어 포함)
   - 성능이 우수하고 범용적

2. cross-encoder/ms-marco-MiniLM-L-6-v2
   - 영어 특화
   - 빠른 처리 속도

3. BAAI/bge-reranker-large
   - 고성능 모델
   - 정확도 우선

4. cross-encoder/ms-marco-TinyBERT-L-2-v2
   - 초경량 모델
   - 실시간 처리에 적합


## 8. 성능 최적화 팁

### 8.1 배치 처리

In [6]:
# 여러 질의를 배치로 처리하는 예시
queries = [
    "LLM이 뭐야?",
    "Transformer 구조 설명해줘"
]

# 각 질의에 대해 검색 수행
for query in queries:
    results = compression_retriever.invoke(query)
    print(f"질의: {query}")
    print(f"- 검색된 문서 수: {len(results)}\n")

질의 1: LLM이 뭐야?
- 검색된 문서 수: 3

질의 2: Transformer 구조 설명해줘
- 검색된 문서 수: 3


### 8.2 캐싱 전략

In [7]:
# 간단한 캐싱 예시
from functools import lru_cache
import hashlib

# 결과 캐싱을 위한 딕셔너리
cache = {}

def get_cached_results(query, retriever):
    """캐싱된 검색 결과 반환"""
    # 질의를 해시로 변환하여 키로 사용
    query_hash = hashlib.md5(query.encode()).hexdigest()
    
    if query_hash not in cache:
        # 캐시에 없으면 검색 수행 후 저장
        cache[query_hash] = retriever.invoke(query)
        print(f"- 새로운 검색 수행 및 캐싱")
    else:
        print(f"- 캐시 히트! 저장된 결과 반환")
    
    return cache[query_hash]

# 캐싱 테스트
print("캐시 사용 예시:")
test_query = "Word2Vec 에 대해서 알려줄래?"
print(f"- 질의: {test_query}")

# 첫 번째 호출 - 캐시에 저장
_ = get_cached_results(test_query, compression_retriever)

# 두 번째 호출 - 캐시에서 반환
cached_results = get_cached_results(test_query, compression_retriever)
print(f"- 결과 문서 수: {len(cached_results)}")

캐시 사용 예시:
- 질의: Word2Vec 에 대해서 알려줄래?
- 캐시 히트! 저장된 결과 반환
- 결과 문서 수: 3


## 9. RAG 시스템에 통합

In [8]:
# RAG 시스템 통합 예시 코드
print("RAG 파이프라인 구성 예시:")
print("""
def rag_pipeline(query):
    # 1. 검색 및 재순위화
    docs = compression_retriever.invoke(query)
    
    # 2. 컨텍스트 생성
    context = "\\n\\n".join([doc.page_content for doc in docs])
    
    # 3. 프롬프트 생성
    prompt = f\"\"\"
    다음 컨텍스트를 참고하여 질문에 답하세요.
    
    컨텍스트:
    {context}
    
    질문: {query}
    답변:
    \"\"\"
    
    # 4. LLM 호출 (여기서는 예시)
    # response = llm.invoke(prompt)
    
    return prompt
""")

RAG 파이프라인 구성 예시:

def rag_pipeline(query):
    # 1. 검색 및 재순위화
    docs = compression_retriever.invoke(query)
    
    # 2. 컨텍스트 생성
    context = "\n\n".join([doc.page_content for doc in docs])
    
    # 3. 프롬프트 생성
    prompt = f"""
    다음 컨텍스트를 참고하여 질문에 답하세요.
    
    컨텍스트:
    {context}
    
    질문: {query}
    답변:
    """
    
    # 4. LLM 호출 (여기서는 예시)
    # response = llm.invoke(prompt)
    
    return prompt


## 10. 요약 및 모범 사례

### Cross Encoder Reranker 사용 시 핵심 포인트

1. **2-Stage Retrieval 패턴 활용**
   - Stage 1: Bi-encoder로 빠르게 후보 추출 (10-20개)
   - Stage 2: Cross-encoder로 정밀 재순위화 (3-5개)

2. **적절한 모델 선택**
   - 한국어: BAAI/bge-reranker-v2-m3
   - 영어: cross-encoder/ms-marco-* 시리즈
   - 속도 우선: TinyBERT 기반 모델
   - 정확도 우선: Large 모델

3. **성능 최적화**
   - 배치 처리로 처리량 향상
   - 결과 캐싱으로 반복 질의 최적화
   - 비동기 처리로 응답 시간 단축

4. **Trade-off 관리**
   - top_n 파라미터로 정확도와 속도 균형 조절
   - 실시간 서비스는 경량 모델 사용
   - 배치 처리는 고성능 모델 사용

5. **모니터링 및 평가**
   - 재순위화 전후 결과 비교
   - 처리 시간 측정
   - 사용자 피드백 수집

## 참고 자료

- [LangChain CrossEncoderReranker Documentation](https://python.langchain.com/docs/integrations/retrievers/cross_encoder_reranker)
- [Hugging Face Cross-Encoder Models](https://huggingface.co/cross-encoder)
- [BAAI BGE Reranker Models](https://huggingface.co/BAAI)
- [MS MARCO Cross-Encoders](https://github.com/UKPLab/sentence-transformers)